In [1]:
import pandas as pd

import json

with open('data/gg2013.json', 'r') as file:
    data = json.load(file)

# Convert the JSON data to a pandas DataFrame
df = pd.DataFrame(data)

# Extract user information into separate columns
df['user_screen_name'] = df['user'].apply(lambda x: x['screen_name'])
df['user_id'] = df['user'].apply(lambda x: x['id'])

# Drop the original 'user' column as we've extracted the needed information
df = df.drop('user', axis=1)

# Convert timestamp_ms to datetime
df['timestamp'] = pd.to_datetime(df['timestamp_ms'], unit='ms')

# Drop the original timestamp_ms column
df = df.drop('timestamp_ms', axis=1)

# Reorder columns for better readability
df = df[['id', 'timestamp', 'user_id', 'user_screen_name', 'text']]

# Display the first few rows of the DataFrame
# print(df.head())



In [2]:
import re

# Function to apply regex patterns and extract potential winners
def extract_winners(text, award):
    # Improved regex to properly handle 'just' variations
    just_variations = r'(?:(?:(?:she|he)\s+)?just\s+)?'
    winner_patterns = [
        r'(\w+(?:\s+\w+)?)\s+' + just_variations + r'wins\s+(?!' + award + ')',
        r'(\w+(?:\s+\w+)?)\s+' + just_variations + r'won\s+(?!' + award + ')',
        r'(\w+(?:\s+\w+)?)\s+' + just_variations + r'awarded\s+(?!' + award + ')',
        r'(\w+(?:\s+\w+)?)\s+' + just_variations + r'receives\s+(?!' + award + ')',
        r'(\w+(?:\s+\w+)?)\s+' + just_variations + r'received\s+(?!' + award + ')'
    ]
    winners = []
    for pattern in winner_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        winners.extend(matches)
    return winners

# Apply the extraction function to the 'text' column
df['potential_winners'] = df['text'].apply(lambda x: extract_winners(x, "Best Picture"))

# Print all non-NaN values in the potential_winners column
all_winners = df['potential_winners'].dropna()
winner_counts = {}
for winners in all_winners:
    if winners:  # Check if the list is not empty
        for winner in winners:
            if winner in winner_counts:
                winner_counts[winner] += 1
            else:
                winner_counts[winner] = 1

# Create the JSON structure
output = {
    "Award": "Best Picture",
    "Nominees": [],  # We don't have nominee information in the current data
    "Presenters": [],  # We don't have presenter information in the current data
    "Winner": [
        {
            "Name": winner,
            "Number of Tweets": count
        } for winner, count in winner_counts.items()
    ]
}

# Sort the winners by number of tweets in descending order
output["Winner"] = sorted(output["Winner"], key=lambda x: x["Number of Tweets"], reverse=True)

# print(json.dumps(output, indent=4))

assume nothing about outputs

from winners list, identify candidates who fit type restrictions

store in nominees - check against 2013 data

Type constraints:
- eliminate pronouns, random phrases

Goal(s):
- Map entities to names/nicknames -> map should contain entity names in some standard format (First, Last) 
- Map entities to some quantity of popularity

Attempt at entity recognition from "winners" list

In [3]:
import spacy

winners = sorted(output["Winner"], key=lambda x: x["Number of Tweets"], reverse=True)

spacy_model = spacy.load('en_core_web_lg') # better entity recognition capability than en_core_web_sm

# {
#     Name : _,
#     Entities : []
# }
entity_list = []

for i in range(len(winners)):
    winner_name = winners[i]["Name"]
    spacy_output = spacy_model(winner_name)
    # print(f"spacy output: {spacy_output.ents}")
    # if spacy_output.ents == (): print("NO ENTITY IDENTIFIED")
    associated_entities = []
    for entity in spacy_output.ents:
        # print(f"entity:{entity}")
        # print([entity.text, entity.label_])
        # entity_list.append(entity.text)
        associated_entities.append(entity.text)
    
    name_entities = {
        "Name" : winner_name,
        "Entities" : associated_entities
    }
    
    entity_list.append(name_entities)
    
for i in range(30): print(entity_list[i])


OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

Define Entity List to check against:
- hard coding testing purposes, eventually will need to extract these too

In [ ]:
# ENTITIES = []

# for entry in entity_list:
#     if len(entry['Entities']) > 0: ENTITIES.append(entry['Entities']) 

ENTITIES = [
    'Ben Affleck', 
    'Anne Hathaway', 
    'Julianne Moore', 
    'Adele', 
    'Jessica Chastain', 
    'Daniel Day-Lewis', 
    'Denzel Washington', 
    'Jonah Hill', 
    'Brad Pitt', 
    'Amy Poehler'
]

**CLUSTERING**

Many names may be associated with a given entity
- Identify names "similar" to the entity (ex: Anne Hathaway - anne hathaway, @annehathaway, etx )
- Note that not every string may be mapped to an entity

Quantifing "similarity" between strings i.e. string distance?
- token overlap -> # times each word in string appears in each defined entity, return highest entity
- loads of string metrics (https://en.wikipedia.org/wiki/String_metric) -> levenshtein, hamming, jaccard, etx (https://www.nltk.org/api/nltk.metrics.html#module-nltk.metrics.distance)
- considerations for string metrics: some metrics require comparison of strings of identical length (ex: Hamming dist.)


Simple Token Overlap
- https://stackoverflow.com/questions/10136077/python-natural-language-processing-for-named-entities

In [ ]:
def token_overlap(query_string, classes):
    """
    Computes the most "likely" class for the given query string.

    First normalises the query to lower case, then computes the number of
    overlapping tokens for each of the possible classes.

    The class(es) with the highest overlap are returned as a list.

    """
    query_tokens = query_string.lower().split() # lowercase query
    class_tokens = [[x.lower() for x in c.split()] for c in classes] # lowercase each class in CLASSES
    # print(f"tokens:{class_tokens}")


    overlap = [0] * len(classes) # num times each word in query string appears for each defined CLASS 
    # check overlap on word/token level, not char
    for token in query_tokens:
        for index in range(len(classes)): 
            if token in class_tokens[index]:
                overlap[index] += 1

    # print(overlap)

    sorted_overlap = [(count, index) for index, count in enumerate(overlap)]
    sorted_overlap.sort()
    sorted_overlap.reverse()

    best_count = sorted_overlap[0][0]

    best_classes = []
    for count, index in sorted_overlap:
        if count == best_count and count > 0: # count > 0 -> DON'T FORCE MAPPING IF NO OVERLAP WITH ANY ENTITY
            best_classes.append(classes[index]) # (classes[index], count) to get token overlap count
        else:
            break

    return best_classes


print(token_overlap("hathaway, anne", classes=ENTITIES))

['Anne Hathaway']


In [ ]:
import nltk
from nltk.metrics import edit_distance, jaccard_distance

str = "chastain, jessica"
dist_measure = 0

for entity in ENTITIES:
    similarity = jaccard_distance(set([x for x in str]), set([x for x in entity]))
    print(f"Entity: {entity} | Similarity: {similarity}")

Entity: Ben Affleck | Similarity: 0.75
Entity: Anne Hathaway | Similarity: 0.6
Entity: Julianne Moore | Similarity: 0.7058823529411765
Entity: Adele | Similarity: 0.9285714285714286
Entity: Jessica Chastain | Similarity: 0.3076923076923077
Entity: Daniel Day-Lewis | Similarity: 0.6470588235294118
Entity: Denzel Washington | Similarity: 0.5294117647058824
Entity: Jonah Hill | Similarity: 0.6666666666666666
Entity: Brad Pitt | Similarity: 0.7333333333333333
Entity: Amy Poehler | Similarity: 0.8333333333333334


Given some list of entities, map names in output to each entity

Aggregate the counts for each name into count for that entity

"Winner" is entity with highest count

In [ ]:
import random
import nltk

# data structure -> entity : [different names]
entity_names = {key: [] for key in ENTITIES}

# data structure -> entity : count
entity_count = {key: 0 for key in ENTITIES} 

winners = sorted(output["Winner"], key=lambda x: x["Number of Tweets"], reverse=True)

# traverse names in winners
for i in range(len(winners)):
    winner_info = winners[i] # name & tweet count
    winner_name = winner_info["Name"]
    winner_count = winner_info["Number of Tweets"]

    # print(winner_info)
    
    # identify entities "closest" to winner_name - replace token_overlap w/ any similarity metric
    candidate_entities = token_overlap(winner_name, classes=ENTITIES)         
        
    # don't map if no entity recognized
    if len(candidate_entities) == 0: continue
    
    # typically single candidate identified, but in case multiple top candidates named pick random - should probably change
    identified_entity = random.choice(candidate_entities) 
    
    # print(f"Name: {winner_name} | Candidate entities: {candidate_entities} | Identified entity: {identified_entity}")

    # map name to entity, update entity count
    entity_names[identified_entity].append(winner_name)
    entity_count[identified_entity] += winner_count

    

# winner = entity w/ highest count
dict(sorted(entity_count.items(), key=lambda item: item[1], reverse=True))
# print(entity_names)

IndentationError: expected an indented block after 'for' statement on line 24 (2402185589.py, line 28)

In [ ]:
entity_names


{'Ben Affleck': ['Ben Affleck',
  'Affleck',
  'Ben affleck',
  'Ben',
  'BEN AFFLECK',
  'if Affleck',
  'ben affleck',
  'Affleck has',
  'So Ben',
  'BEN AFLECK',
  'when Affleck',
  'Ben Afleck',
  'director Affleck',
  'Ben just',
  'happy Ben',
  'that Affleck',
  'affleck',
  'ben afleck',
  'Affleck just',
  'glad Affleck',
  'Ben Afflect',
  'Glad Ben',
  'thrilled Affleck',
  'Affleck also'],
 'Anne Hathaway': ['Anne Hathaway',
  'ANNE HATHAWAY',
  'Hathaway both',
  'ANNE',
  'pretend Hathaway',
  'anne hathaway',
  'Hathaway',
  'Anne Hathway',
  'Anne Hatheway',
  'anne hatheway',
  'Anne',
  'Amne Hathaway',
  'girl Anne',
  'Hathaway to',
  'Hathaway deserve',
  'Hathaway immediately',
  'and Anne',
  'Hathaway have'],
 'Julianne Moore': ['Julianne Moore',
  'Julianna Moore',
  'Julianne',
  'Moore 2',
  'Julian Moore',
  'JULIANNE MOORE',
  'now Julianne'],
 'Adele': ['Adele',
  'when Adele',
  'by Adele',
  'that Adele',
  'adele',
  'mum Adele',
  'glad Adele',
  'hop

Aggregation ideas)
- consider proximity in text: words/names mentioned in close succession are likely related
- entities are of a certain type: necessary for type checking

Incorporate type checking in entity mapping:
- how to implement?
    - what to check correct "type" against? List of awards/movies/people?
    - these lists can be imported into the repo? use external database? 
- 

Keywords associated with different roles?
- Hosts: starting, hosting
- Awards: best (assumed)

Mining award names:
- syntactic parsing

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

df[df['cleaned_text'].str.contains('best')]['cleaned_text']

# df[df[['cleaned_text']].str.contains('hosting')]

In [16]:
import os

# os.path.abspath('')

os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'gg2013answers.json')

'c:\\Users\\jimin\\OneDrive\\Documents\\Northwestern\\Fall 2024\\NLP\\Projects\\TweetMiner\\data\\gg2013answers.json'

Extracting award names:
- Identify categories - what does that mean? Need to extract entirety of award name?

In [17]:
import spacy
import json
import nltk
from pathlib import Path
import os

# df = pd.read_csv(Path(globals()['_dh'][0])/'Data'/'csvFile')
gg2013answers_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'gg2013answers.json')

# Open and read the JSON file
with open(gg2013answers_path, 'r') as file:
    data = json.load(file)

data

{'hosts': ['amy poehler', 'tina fey'],
 'award_data': {'best screenplay - motion picture': {'nominees': ['zero dark thirty',
    'lincoln',
    'silver linings playbook',
    'argo'],
   'presenters': ['robert pattinson', 'amanda seyfried'],
   'winner': 'django unchained'},
  'best director - motion picture': {'nominees': ['kathryn bigelow',
    'ang lee',
    'steven spielberg',
    'quentin tarantino'],
   'presenters': ['halle berry'],
   'winner': 'ben affleck'},
  'best performance by an actress in a television series - comedy or musical': {'nominees': ['zooey deschanel',
    'tina fey',
    'julia louis-dreyfus',
    'amy poehler'],
   'presenters': ['aziz ansari', 'jason bateman'],
   'winner': 'lena dunham'},
  'best foreign language film': {'nominees': ['the intouchables',
    'kon tiki',
    'a royal affair',
    'rust and bone'],
   'presenters': ['arnold schwarzenegger', 'sylvester stallone'],
   'winner': 'amour'},
  'best performance by an actor in a supporting role in a

In [22]:
import spacy
import json
import nltk
from nltk.tokenize import word_tokenize
from pathlib import Path

# df = pd.read_csv(Path(globals()['_dh'][0])/'Data'/'csvFile')
gg2013answers_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'gg2013answers.json')

# Open and read the JSON file
with open(gg2013answers_path, 'r') as file:
    data = json.load(file)

awards_list = list(data['award_data'].keys())

spacy_model = spacy.load('en_core_web_sm')

# vary input statement w/ different award names
# check recognition performance
for award in awards_list:
    print(f"AWARD: {award}")
    input_sentence = 'I just heard Ben Affeck won ' + str(award) + ' and he was beyond grateful!'

    text = word_tokenize(input_sentence)
    print(nltk.pos_tag(text))
    
    # spacy_output = spacy_model(input_sentence)

    # for chunk in spacy_output.noun_chunks:
    #     print([chunk.text, chunk.root.head.text])
    # print('\n')

    # for token in spacy_output:
    #     print(
    #     token.text,
    #     token.dep_,
    #     token.head.text,
    #     token.head.pos_,
    #     [child for child in token.children]
    #     )

AWARD: best screenplay - motion picture
[('I', 'PRP'), ('just', 'RB'), ('heard', 'VBD'), ('Ben', 'NNP'), ('Affeck', 'NNP'), ('won', 'VBD'), ('best', 'JJS'), ('screenplay', 'NN'), ('-', ':'), ('motion', 'NN'), ('picture', 'NN'), ('and', 'CC'), ('he', 'PRP'), ('was', 'VBD'), ('beyond', 'IN'), ('grateful', 'JJ'), ('!', '.')]
AWARD: best director - motion picture
[('I', 'PRP'), ('just', 'RB'), ('heard', 'VBD'), ('Ben', 'NNP'), ('Affeck', 'NNP'), ('won', 'VBD'), ('best', 'JJS'), ('director', 'NN'), ('-', ':'), ('motion', 'NN'), ('picture', 'NN'), ('and', 'CC'), ('he', 'PRP'), ('was', 'VBD'), ('beyond', 'IN'), ('grateful', 'JJ'), ('!', '.')]
AWARD: best performance by an actress in a television series - comedy or musical
[('I', 'PRP'), ('just', 'RB'), ('heard', 'VBD'), ('Ben', 'NNP'), ('Affeck', 'NNP'), ('won', 'VBD'), ('best', 'JJS'), ('performance', 'NN'), ('by', 'IN'), ('an', 'DT'), ('actress', 'NN'), ('in', 'IN'), ('a', 'DT'), ('television', 'NN'), ('series', 'NN'), ('-', ':'), ('comedy'

In [23]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Type checking movie vs. human awards)
- just check whether candidate entity is human

Type checking against movies/credits data)
- data contains many movies w/ cast & crew for each of those movies
- winners/nominees must be entities that exist in the cast/crew for those movies
    - extract cast for each movie (top N?) and store in db
    - check against this list when identifying entities


In [48]:
import pandas as pd

movies_metadata_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'movies_metadata.csv')
credits_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'credits.csv')

movies_df = pd.read_csv(movies_metadata_path)
credits_df = pd.read_csv(credits_path)


C:\Users\jimin\AppData\Local\Temp\ipykernel_8944\1790623868.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(movies_metadata_path)


In [51]:
movies_df['status'].value_counts()

status
Released           45014
Rumored              230
Post Production       98
In Production         20
Planned               15
Canceled               2
Name: count, dtype: int64

In [53]:
import pandas as pd
import ast
import os

# create combined df w/ movies & credits
# FILTER: for 'Released' & relevant year
def create_movies_credits_df(year):
    # declare file paths
    movies_metadata_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'movies_metadata.csv')
    credits_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'credits.csv')

    movies = pd.read_csv(movies_metadata_path)
    credits = pd.read_csv(credits_path)

    # filter to 'Released' movies only
    movies = movies[movies['status']=='Released']
    
    # remove unnecessary columns
    movies.drop(columns=['belongs_to_collection', 'budget', 'homepage', 'imdb_id', 'overview', 'poster_path', 'runtime', 'status', 'tagline', 'video'], inplace=True)
    # function to check int id types
    def is_integer(val):
        try:
            # try to convert to int
            int(val)
            return True
        except (ValueError, TypeError):
            return False

    # filter rows where 'id' is an integer-like value
    movies = movies[movies.id.apply(is_integer)]

    # convert 'id' column to int
    movies.id = movies.id.astype(int)

    # merge with credits df
    df = pd.merge(movies, credits, on='id')
    df.drop(columns=['id'], inplace=True)

    # clean columns
    cols = ['genres', 'production_companies', 'production_countries', 'spoken_languages']
    for col in cols:
        df[col] = df[col].apply(extract_names)
    df.release_date = pd.to_datetime(df.release_date)
    df.cast = df.cast.apply(clean_cast_data)
    df.crew = df.crew.apply(clean_crew_data)

    # filter movie/credit data for relevant year
    df = df[df['release_date'].dt.year == year]

    return df


# extract the category names
def extract_names(name_str):
    if pd.isna(name_str):
        return []
    # convert the string representation of the list to an actual list
    str_list = ast.literal_eval(name_str)
    # extract the 'name' from each dictionary in the list
    names = [i['name'] for i in str_list]
    # return list of names as a string
    return ', '.join(names)

# clean the cast data
def clean_cast_data(cast_str):
    # convert string representation of the list to an actual list
    cast_list = ast.literal_eval(cast_str)

    # extract relevant fields and change gender values
    cleaned_cast = []
    for member in cast_list:
        cleaned_member = {
            'character': member['character'],
            'gender': 'm' if member['gender'] == 2 else 'f' if member['gender'] == 1 else None,
            'name': member['name'],
            'order': member['order']
        }
        cleaned_cast.append(cleaned_member)
    return cleaned_cast

# clean the crew data
def clean_crew_data(crew_str):
    # convert string representation of the list to an actual list
    crew_list = ast.literal_eval(crew_str)

    # extract relevant fields
    cleaned_crew = []
    for member in crew_list:
        cleaned_member = {
            'job': member['job'],
            'name': member['name']
        }
        cleaned_crew.append(cleaned_member)
    return cleaned_crew

In [54]:
import sys

# setting path
# sys.path.append('../util_functions')

# importing
# from util_functions.movie_data_utils import create_movies_credits_df

movies_credits_df = create_movies_credits_df('2013')

movies_credits_df.head()

C:\Users\jimin\AppData\Local\Temp\ipykernel_8944\769975708.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(movies_metadata_path)


,adult,genres,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,title,vote_average,vote_count,cast,crew
1079,False,"Comedy, Horror",en,The Sleepover,0.135596,,,2013-10-12,0.0,,The Sleepover,8.0,1.0,"[{'character': 'Rachel', 'gender': None, 'name...","[{'job': 'Director', 'name': 'Chris Cullari'}]"
4341,False,Drama,fr,Camille Claudel 1915,0.134014,"Canal+, Arte France Cinéma, 3B Productions, C....",France,2013-03-13,115860.0,Français,Camille Claudel 1915,7.0,20.0,"[{'character': 'Camille Claudel', 'gender': 'f...","[{'job': 'Director', 'name': 'Bruno Dumont'}, ..."
4342,False,Drama,fr,Camille Claudel 1915,0.134014,"Canal+, Arte France Cinéma, 3B Productions, C....",France,2013-03-13,115860.0,Français,Camille Claudel 1915,7.0,20.0,"[{'character': 'Camille Claudel', 'gender': 'f...","[{'job': 'Producer', 'name': 'Rachid Bouchareb..."
11215,False,Drama,en,La Cicatrice,0.025665,Cicatrice Film inc,Canada,2013-03-03,0.0,Français,The Scar,0.0,0.0,"[{'character': 'Richard Tremblay', 'gender': '...","[{'job': 'Director', 'name': 'Jimmy Larouche'}..."
14724,False,Animation,en,Dante's Hell Animated,0.787493,Dante's Inferno Animation,United States of America,2013-11-25,0.0,Italiano,Dante's Hell Animated,8.0,4.0,"[{'character': 'Circles Introduction (voice)',...","[{'job': 'Director', 'name': 'Boris Acosta'}]"


Check movie cast & crew data

Create DB with top N actors for each movie

Check against this DB when identifying relevant entities

In [55]:
movies_credits_df.columns

Index(['adult', 'genres', 'original_language', 'original_title', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'spoken_languages', 'title', 'vote_average', 'vote_count',
       'cast', 'crew'],
      dtype='object')